In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import tensorflow as tf
import numpy as np
import os

In [2]:
cwd = os.getcwd()
print(cwd)

D:\Language Identifier\Notebook


In [3]:
X_input = np.load("D:/Language Identifier/Data/input_X.npy")
Y_input = np.load("D:/Language Identifier/Data/input_Y.npy")

In [5]:
x_train = X_input[:25000]
x_test = X_input[25000:30000]
y_train = Y_input[:25000]
y_test = Y_input[25000:30000]

In [6]:
model = Sequential()

model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs = 15, batch_size = 32,
          validation_data=(x_test, y_test))

Epoch 1/15
782/782 [==============================] - 27s 34ms/step - loss: 1.1794 - accuracy: 0.4850 - val_loss: 0.7734 - val_accuracy: 0.7248
Epoch 2/15
782/782 [==============================] - 25s 33ms/step - loss: 0.8957 - accuracy: 0.7104 - val_loss: 0.5885 - val_accuracy: 0.8092
Epoch 3/15
782/782 [==============================] - 26s 33ms/step - loss: 0.5715 - accuracy: 0.8060 - val_loss: 0.4069 - val_accuracy: 0.8614
Epoch 4/15
782/782 [==============================] - 26s 33ms/step - loss: 0.4345 - accuracy: 0.8573 - val_loss: 0.3194 - val_accuracy: 0.8938
Epoch 5/15
782/782 [==============================] - 26s 34ms/step - loss: 0.6436 - accuracy: 0.8292 - val_loss: 0.3624 - val_accuracy: 0.8734
Epoch 6/15
782/782 [==============================] - 26s 33ms/step - loss: 0.3725 - accuracy: 0.8777 - val_loss: 0.2847 - val_accuracy: 0.9032
Epoch 7/15
782/782 [==============================] - 26s 33ms/step - loss: 0.3082 - accuracy: 0.9012 - val_loss: 0.2697 - val_accuracy:

In [9]:
model.save("/Volumes/Henry's/Language Identifier/Model/model1.model")

INFO:tensorflow:Assets written to: /usr/local/NNShared/hali/Model/model1.model/assets


In [54]:
### Applications ###

new_model = tf.keras.models.load_model("/Volumes/Henry's/Language Identifier/Model/model1.model")
prediction = new_model.predict([x_test])

print(x_test[6])
print(y_test[6])
print(np.argmax(prediction[6]))

[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[0 0 1 0 0 0]
2


In [4]:
def language_detector(sentence):
    if len(sentence) < 30:
        print('sentence is too short. Try a longer one.')
    else:
        sentence = sentence[:30].lower()
        test =  np.array([np.reshape(string_vectorizer(sentence, dic),(len(sentence),len(dic)))])
        prediction = np.argmax(new_model.predict([test]))
        
        if prediction == 0:
            print('It is English')
        elif prediction == 4:
            print('It is French')
        elif prediction == 5:
            print('It is Finnish')
        elif prediction == 2:
            print('It is Dutch')
        elif prediction == 3:
            print('It is German')
        elif prediction == 1:
            print('It is Portuguese')
            
def string_vectorizer(strng, dic):
    vector = [[0 if char != letter.lower() else 1 for char in dic] 
                for letter in strng]
    return vector

In [5]:
import json
import tensorflow as tf
import numpy as np

dic = json.load(open("/Volumes/Henry's/Language Identifier/Data/characterdic.json", 'r'))
new_model = tf.keras.models.load_model("/Volumes/Henry's/Language Identifier/Model/model1.model")

2021-12-06 11:54:29.574599: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
print('Enter your sentence:')
sentence = input()
language_detector(sentence)

Enter your sentence:
Mais pas seulement ! Le français a aussi été influencé par la langue de ses pays voisins : l'anglais, l'espagnol, l'italien, l'arabe, etc.
It is French
